In [1]:
import os
import sys
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from tqdm import tqdm

import seaborn as sns
sns.set_theme(style="whitegrid")

sys.path.append("../..")
from src.nft_analytics import NFTAnalytics
from src.infura_api import InfuraAPI

## CBD details

In [2]:
details = {
  "collection": {
    "banner_image_url": "https://lh3.googleusercontent.com/hqd6l6lIfmWWO47zZBRd7lLgvukjLgQ38z2axqKM7rr-PPm0EYmSkUoL9346hEseNh93mrBE7BFz6FIWfhIfd55Q_k2eiy_54bwvCg=s2500",
    "chat_url": None,
    "created_date": "2021-08-20T17:40:26.243798",
    "default_to_fiat": False,
    "description": "Chibi Dinos live to play ball. Pick the highest IQ or the freshest Chibi for your chance at glory. The 10,000 Chibis live on planet Hoop and will defend it with their lives, so watch your step!\n",
    "dev_buyer_fee_basis_points": "0",
    "dev_seller_fee_basis_points": "500",
    "discord_url": "https://discord.gg/dinos",
    "display_data": {
      "card_display_style": "cover"
    },
    "external_url": "https://chibidinos.io",
    "featured": False,
    "featured_image_url": "https://lh3.googleusercontent.com/DX7HIdeoVnepFvZDhK0s_FiV7yfkwpgXgZG2PyS2bnQ6Dco7lPIOH5xY3UAciDWKtU49kU7GUXbSfjwUgPgZGNdGZudA2EdZSwZ0f7E=s300",
    "hidden": False,
    "safelist_request_status": "verified",
    "image_url": "https://lh3.googleusercontent.com/EM6pjQ97QydB-nrCWrKA6pdOxiBPfsv3O5DOqfiJqKjLSxWO2djfPt-8LJQrmGFi7on3dXRwgW8L66WoBrnakoFxmmH-AUK7YwDd=s120",
    "is_subject_to_whitelist": False,
    "large_image_url": "https://lh3.googleusercontent.com/DX7HIdeoVnepFvZDhK0s_FiV7yfkwpgXgZG2PyS2bnQ6Dco7lPIOH5xY3UAciDWKtU49kU7GUXbSfjwUgPgZGNdGZudA2EdZSwZ0f7E=s300",
    "medium_username": None,
    "name": "CHIBI DINOS",
    "only_proxied_transfers": False,
    "opensea_buyer_fee_basis_points": "0",
    "opensea_seller_fee_basis_points": "250",
    "payout_address": "0x15aaf062eff5c48dcbaa98d8c7979d7d2d8636d1",
    "require_email": False,
    "short_description": None,
    "slug": "chibidinos",
    "telegram_url": None,
    "twitter_username": "dinoschibi",
    "instagram_username": "chibidinosnft/",
    "wiki_url": None,
    "is_nsfw": False
  },
  "address": "0xe12edaab53023c75473a5a011bdb729ee73545e8",
  "asset_contract_type": "non-fungible",
  "created_date": "2021-08-20T17:18:03.694053",
  "name": "CHIBI DINOS",
  "nft_version": "3.0",
  "opensea_version": None,
  "owner": 284611912,
  "schema_name": "ERC721",
  "symbol": "CHIBIDINOS",
  "total_supply": "0",
  "description": "Chibi Dinos live to play ball. Pick the highest IQ or the freshest Chibi for your chance at glory. The 10,000 Chibis live on planet Hoop and will defend it with their lives, so watch your step!\n",
  "external_link": "https://chibidinos.io",
  "image_url": "https://lh3.googleusercontent.com/EM6pjQ97QydB-nrCWrKA6pdOxiBPfsv3O5DOqfiJqKjLSxWO2djfPt-8LJQrmGFi7on3dXRwgW8L66WoBrnakoFxmmH-AUK7YwDd=s120",
  "default_to_fiat": False,
  "dev_buyer_fee_basis_points": 0,
  "dev_seller_fee_basis_points": 500,
  "only_proxied_transfers": False,
  "opensea_buyer_fee_basis_points": 0,
  "opensea_seller_fee_basis_points": 250,
  "buyer_fee_basis_points": 0,
  "seller_fee_basis_points": 750,
  "payout_address": "0x15aaf062eff5c48dcbaa98d8c7979d7d2d8636d1"
}

In [3]:
import requests
import json
from pycoingecko import CoinGeckoAPI

In [4]:
collection_slug = "chibidinos"
url = f"https://api.opensea.io/api/v1/collection/{collection_slug}"

response = requests.get(url, headers={"Accept": "application/json"})
json_response = json.loads(response.text)

In [5]:
cg = CoinGeckoAPI()
eth_price = float(cg.get_price(ids="ethereum", vs_currencies="usd")["ethereum"]["usd"])

In [6]:
stats = json_response["collection"]["stats"]
pprint(stats)

{'average_price': 0.13089395537572857,
 'count': 10000.0,
 'floor_price': 0.099,
 'market_cap': 1339.6161971830986,
 'num_owners': 3200,
 'num_reports': 5,
 'one_day_average_price': 0.13049999999999998,
 'one_day_change': -0.626476443686832,
 'one_day_sales': 18.0,
 'one_day_volume': 2.3489999999999998,
 'seven_day_average_price': 0.13396161971830986,
 'seven_day_change': 1.7299863002335532,
 'seven_day_sales': 142.0,
 'seven_day_volume': 19.02255,
 'thirty_day_average_price': 0.1242395179138321,
 'thirty_day_change': 0.2253279230055001,
 'thirty_day_sales': 441.0,
 'thirty_day_volume': 54.78962739999996,
 'total_sales': 15451.0,
 'total_supply': 10000.0,
 'total_volume': 2022.442504510382}


In [7]:
df_seven_day_metrics = pd.DataFrame(columns=["Seven Day Metrics", "Value"])
df_seven_day_metrics["Seven Day Metrics"] = ["Volume", "Change", "Average Price", "Sales"]
df_seven_day_metrics["Value"] = [
    f'Ξ{stats["seven_day_volume"]:.1f} (${eth_price * stats["seven_day_volume"]:.0f})', 
    f'Ξ{stats["seven_day_change"]:.2f} (${eth_price * stats["seven_day_change"]:.0f})', 
    f'Ξ{stats["seven_day_average_price"]:.2f} (${eth_price * stats["seven_day_average_price"]:.0f})',
    f'{int(stats["seven_day_sales"])}', 

]
df_seven_day_metrics

,Seven Day Metrics,Value
0,Volume,Ξ19.0 ($56442)
1,Change,Ξ1.73 ($5133)
2,Average Price,Ξ0.13 ($397)
3,Sales,142


In [8]:
df_thirty_day_metrics = pd.DataFrame(columns=["Thirty Day Metrics", "Value"])
df_thirty_day_metrics["Thirty Day Metrics"] = ["Volume", "Change", "Average Price", "Sales"]
df_thirty_day_metrics["Value"] = [
    f'Ξ{stats["thirty_day_volume"]:.1f} (${eth_price * stats["thirty_day_volume"]:,.0f})', 
    f'Ξ{stats["thirty_day_change"]:.2f} (${eth_price * stats["thirty_day_change"]:,.0f})', 
    f'Ξ{stats["thirty_day_average_price"]:.2f} (${eth_price * stats["thirty_day_average_price"]:,.0f})',
    f'{int(stats["thirty_day_sales"]):,}', 

]
df_thirty_day_metrics

,Thirty Day Metrics,Value
0,Volume,"Ξ54.8 ($162,566)"
1,Change,Ξ0.23 ($669)
2,Average Price,Ξ0.12 ($369)
3,Sales,441


In [9]:
df_overall_metrics = pd.DataFrame(columns=["Overall Metrics", "Value"])
df_overall_metrics["Overall Metrics"] = ["Market Cap", "Total Volume", "Floor Price", "Average Price", "Total Sales", "Hodlers"]
df_overall_metrics["Value"] = [
    f'Ξ{stats["market_cap"]:.1f} (${eth_price * stats["market_cap"]:,.0f})', 
    f'Ξ{stats["total_volume"]:.1f} (${eth_price * stats["total_volume"]:,.0f})', 
    f'Ξ{stats["floor_price"]:.2f} (${eth_price * stats["floor_price"]:,.0f})', 
    f'Ξ{stats["average_price"]:.2f} (${eth_price * stats["average_price"]:,.0f})', 
    f'{int(stats["total_sales"]):,}',
    f'{int(stats["num_owners"]):,}',
]
df_overall_metrics

,Overall Metrics,Value
0,Market Cap,"Ξ1339.6 ($3,974,762)"
1,Total Volume,"Ξ2022.4 ($6,000,769)"
2,Floor Price,Ξ0.10 ($294)
3,Average Price,Ξ0.13 ($388)
4,Total Sales,"15,451"
5,Hodlers,"3,200"


In [11]:
page_title_text='Chibi Dinos Report'
title_text = 'Chibi Dinos Report'
stats_text = 'Important stats'
table_settings = dict(header=False, index=False, classes="mystyle", col_space=5)

html = f'''
<html>
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <head>
        <title>{page_title_text}</title>
    </head>
    <link rel="stylesheet" type="text/css" href="df_style.css"/>
    <body>
        <h1>{title_text}</h1>
        <p>{details["collection"]["description"]}</p>
        <img src="chibi_dinos.jpeg" width="200">
        <h2>Seven Day Stats</h2>
        {df_seven_day_metrics.to_html(**table_settings)}
        <h2>Thirty Day Stats</h2>
        {df_thirty_day_metrics.to_html(**table_settings)}
        <h2>Overall Stats</h2>
        {df_overall_metrics.to_html(**table_settings)}
    </body>
</html>
'''

with open('chibi_report.html', 'w') as f:
    f.write(html)